In [0]:
-- ============================================
-- DATABRICKS AI FUNCTIONS SHOWCASE
-- Table: users.guy_livni.incoming_radiology_reports_findings
-- ============================================

-- 1. AI_SUMMARIZE: Generate concise summaries of radiology reports
-- Use Case: Quick review dashboards, executive summaries, patient communications
-- This creates a 50-word summary of each report for faster triage
SELECT 
  report_id,
  patient_id,
  referring_physician,
  ai_summarize(raw_text, 50) AS report_summary,
  triage_category
FROM users.guy_livni.incoming_radiology_reports_findings
LIMIT 5;

-- 2. AI_EXTRACT: Extract specific medical entities from reports
-- Use Case: Structured data extraction, building medical knowledge graphs, automated coding
-- This pulls out key medical information from unstructured text
SELECT 
  report_id,
  ai_extract(raw_text, ARRAY('diagnosis', 'body part examined', 'contrast usage', 'abnormal findings')) AS extracted_entities,
  raw_text
FROM users.guy_livni.incoming_radiology_reports_findings
LIMIT 5;

-- 3. AI_CLASSIFY: Categorize reports by urgency level
-- Use Case: Automated triage, priority routing, workload distribution
-- This automatically classifies urgency to help prioritize critical cases
SELECT 
  report_id,
  patient_id,
  ai_classify(raw_text, ARRAY('Critical - Immediate Action', 'Urgent - Same Day', 'Routine - Standard Queue', 'Follow-up Required')) AS ai_urgency_classification,
  triage_category AS current_triage,
  ground_truth_critical
FROM users.guy_livni.incoming_radiology_reports_findings
LIMIT 5;

-- 4. AI_ANALYZE_SENTIMENT: Detect concerning language patterns
-- Use Case: Quality assurance, identifying reports needing physician review
-- This analyzes the tone to detect urgency or concern in the language
SELECT 
  report_id,
  ai_analyze_sentiment(raw_text) AS report_tone,
  raw_text
FROM users.guy_livni.incoming_radiology_reports_findings
LIMIT 5;

-- 5. AI_MASK: Protect patient privacy by masking sensitive information
-- Use Case: Data sharing, research datasets, training environments, HIPAA compliance
-- This automatically redacts PII like names, dates, and identifiers
SELECT 
  report_id,
  ai_mask(patient_name, array("name")) AS anonymized_name,
  ai_mask(raw_text, array("exammed organ", "any organ", "entire history")) AS anonymized_report,
  raw_text AS original_report
FROM users.guy_livni.incoming_radiology_reports_findings
LIMIT 3;

-- 6. AI_GEN: Generate follow-up recommendations based on findings
-- Use Case: Clinical decision support, standardizing care protocols, physician assistance
-- This creates actionable next steps for critical findings
SELECT 
  report_id,
  patient_id,
  raw_text,
  ai_gen(CONCAT('Based on this radiology report, suggest appropriate follow-up actions: ', raw_text)) AS follow_up_recommendations,
  triage_category
FROM users.guy_livni.incoming_radiology_reports_findings
WHERE triage_category = 'Critical - Immediate Notification'
LIMIT 3;

-- 7. AI_SIMILARITY: Find similar reports for case comparison
-- Use Case: Clinical research, finding precedent cases, quality benchmarking
-- This identifies reports with similar findings to support clinical decision-making
WITH reference_report AS (
  SELECT raw_text AS reference_text
  FROM users.guy_livni.incoming_radiology_reports_findings
  WHERE report_id = '9aba6a7b-c70a-4af1-892f-4fe44d6ed439'
)
SELECT 
  r.report_id,
  r.patient_id,
  ai_similarity(r.raw_text, ref.reference_text) AS similarity_score,
  r.raw_text, ref.reference_text,
  r.triage_category
FROM users.guy_livni.incoming_radiology_reports_findings r
CROSS JOIN reference_report ref
WHERE r.report_id != '9aba6a7b-c70a-4af1-892f-4fe44d6ed439'
ORDER BY similarity_score DESC
LIMIT 5;


-- 8. AI_QUERY: Query foundation models for custom analysis
-- Use Case: Leverage specific models like Gemini for advanced medical reasoning
-- This uses databricks-gemini-2-5-flash to provide detailed clinical interpretation
SELECT 
  report_id,
  patient_id,
  ai_query(
    'databricks-gemini-2-5-flash',
    CONCAT('You are a radiologist assistant. Analyze this report and identify: 1) Critical findings requiring immediate attention, 2) Recommended imaging follow-up, 3) Differential diagnoses to consider. Report: ', raw_text)
  ) AS gemini_clinical_analysis,
  triage_category
FROM users.guy_livni.incoming_radiology_reports_findings
WHERE ground_truth_critical = true
LIMIT 3;

-- 9. AI_TRANSLATE: Translate reports to Spanish for multilingual care teams
-- Use Case: International hospitals, Spanish-speaking patients, multilingual documentation
-- This translates radiology reports to Spanish for broader accessibility
SELECT 
  report_id,
  patient_id,
  referring_physician,
  raw_text AS original_english_report,
  ai_translate(raw_text, 'es') AS spanish_report
FROM users.guy_livni.incoming_radiology_reports_findings
LIMIT 5;